# 🌞 Solar Rooftop Analysis Tool (Google Cloud Vision API + Rules)

This Google Colab notebook analyzes a rooftop image using Google Cloud Vision API and provides basic solar panel recommendations based on detected labels.

In [ ]:
# 📦 Step 1: Install required packages
!pip install -q requests

In [ ]:
# 📤 Step 2: Upload a satellite rooftop image
from google.colab import files
uploaded = files.upload()

from PIL import Image
import IPython.display as display

image_path = next(iter(uploaded))
image = Image.open(image_path)
display.display(image)

In [ ]:
# 🔍 Step 3: Analyze image using Google Cloud Vision API
import base64
import requests

# Replace with your actual API key
api_key = "YOUR_GOOGLE_CLOUD_VISION_API_KEY"

# Encode image
with open(image_path, "rb") as image_file:
    content = base64.b64encode(image_file.read()).decode()

# Prepare request body
url = f"https://vision.googleapis.com/v1/images:annotate?key={api_key}"
body = {
  "requests": [
    {
      "image": {"content": content},
      "features": [{"type": "LABEL_DETECTION", "maxResults": 10}]
    }
  ]
}

# Send request
response = requests.post(url, json=body)
labels = response.json()['responses'][0]['labelAnnotations']

print("\n🔎 Detected Labels:")
for label in labels:
    print(f"{label['description']} - Confidence: {round(label['score']*100, 2)}%")

In [ ]:
# ⚙️ Step 4: Generate solar recommendations based on label rules
def get_solar_recommendation(labels):
    label_texts = [l['description'].lower() for l in labels]
    
    if "roof" in label_texts and "flat" in label_texts:
        return {
            "panel_type": "Mono PERC",
            "count": 10,
            "area_required": "150 sq ft",
            "estimated_cost": "₹1.5 Lakh",
            "roi_years": "4-5 years"
        }
    elif "roof" in label_texts and "sloped" in label_texts:
        return {
            "panel_type": "Polycrystalline",
            "count": 6,
            "area_required": "90 sq ft",
            "estimated_cost": "₹1 Lakh",
            "roi_years": "5-6 years"
        }
    elif "metal" in label_texts:
        return {
            "panel_type": "Thin Film",
            "count": 8,
            "area_required": "120 sq ft",
            "estimated_cost": "₹1.2 Lakh",
            "roi_years": "4 years"
        }
    else:
        return {
            "panel_type": "Need manual inspection",
            "count": "-",
            "area_required": "-",
            "estimated_cost": "-",
            "roi_years": "-"
        }

recommendation = get_solar_recommendation(labels)

print("\n⚡ Solar Recommendation:")
for key, value in recommendation.items():
    print(f"{key}: {value}")